sqlparse ：sqlparse是一个功能强大的SQL解析器和格式化器，可以解析SQL语句并将其转换为Python对象表示形式。它可以处理各种SQL语句，包括复杂的嵌套查询、多语句等。此外，sqlparse还提供了将SQL语句格式化为易于阅读的形式的功能。

官方网站：https://github.com/andialbrecht/sqlparse

antlr4：antlr4是一个强大的语言识别器生成器，可以根据给定的语法规则生成解析器。它支持广泛的语言和语法，包括SQL。虽然antlr4本身是用Java编写的，但是它也提供了Python的运行时库，可以在Python中使用antlr4生成的解析器。

官方网站：https://www.antlr.org/

PyParsing：PyParsing是一个纯Python实现的解析器库，用于构建解析器和语法分析器。虽然它没有antlr4那么强大，但它简单易用，适合处理简单到中等复杂度的语法结构。

官方网站：https://pyparsing.pyparsing.org/

sqlglot

官网：https://sqlglot.com/sqlglot.html#examples

ast 抽象语法树入门：https://github.com/tobymao/sqlglot/blob/main/posts/ast_primer.md

In [1]:
!pip install sqlglot

  Obtaining dependency information for sqlglot from https://files.pythonhosted.org/packages/fd/1b/d5a8755649f8bcdf2a27af87506d9da2058257c07e82bbc3c4bbae6ca403/sqlglot-21.1.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/363.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/363.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/363.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/363.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/363.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/363.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/363.6 kB ? eta -:--:--
   - -------------------------------------- 10.2/363.6 kB ? eta -:--:--
   - -------------------------------------- 10.2/363.6 kB ? eta -:--:--
   - -------------------------------------- 10.2/363.6 kB ? eta -:--:--
   - -------------------------------------- 10.2/363.6 kB ? eta -:--:--
   - ----------


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# sql语法解析器
import sqlglot
# sql 类型
from sqlglot import exp

In [8]:
sql='''
insert into test
(
    id,
    name
)
select
    t1.id,
    case when nvl(length(t1.name1),0) > 0 then t1.bname1
    else t2.name2 end as name
from table1 t1
left join table2 t2 on t1.id = t2.id
left join (select id from tc3 left join tc4 on tc3.id = tc4.id and tc4.name<>'') t3 on t1.id = t3.id
where id is not null
group by name
'''

In [9]:
# 转换sql为 ast tree
ast=sqlglot.parse_one(sql)

In [15]:
# 获取insert table 名称
for pre in ast.this.find_all(exp.Table):
    print(pre)

test


In [16]:
# 获取from table 名称
for pre in ast.expression.find_all(exp.Table):
    print(pre)

table1 AS t1
table2 AS t2
tc3
tc4


In [6]:
for pre in ast.expression.find_all(exp.Join):
    print(pre)
    for pre_table in pre.find_all(exp.Table):
        print(pre_table)

LEFT JOIN table2 AS t2 ON t1.id = t2.id
table2 AS t2
LEFT JOIN (SELECT id FROM tc3 LEFT JOIN tc4 ON tc3.id = tc4.id AND tc4.name <> '') AS t3 ON t1.id = t3.id
tc3
tc4
LEFT JOIN tc4 ON tc3.id = tc4.id AND tc4.name <> ''
tc4


ast.this

In [26]:
# root 的 this 指
# 如果语句 是 insert 那么 this 指向 insert ... 到 from或select之间的部分
# 如果语句 是 select 那么 this 指向 select ... 到 from之间的部分
ast.this

Schema(
  this=Table(
    this=Identifier(this=test, quoted=False)),
  expressions=[
    Identifier(this=id, quoted=False),
    Identifier(this=name, quoted=False)])

ast.expression

In [27]:
# 指 子查询 或 form 之后的部分
ast.expression

Select(
  expressions=[
    Column(
      this=Identifier(this=id, quoted=False),
      table=Identifier(this=t1, quoted=False)),
    Alias(
      this=Case(
        ifs=[
          If(
            this=GT(
              this=Coalesce(
                this=Length(
                  this=Column(
                    this=Identifier(this=name1, quoted=False),
                    table=Identifier(this=t1, quoted=False))),
                expressions=[
                  Literal(this=0, is_string=False)]),
              expression=Literal(this=0, is_string=False)),
            true=Column(
              this=Identifier(this=bname1, quoted=False),
              table=Identifier(this=t1, quoted=False)))],
        default=Column(
          this=Identifier(this=name2, quoted=False),
          table=Identifier(this=t2, quoted=False))),
      alias=Identifier(this=name, quoted=False))],
  from=From(
    this=Table(
      this=Identifier(this=table1, quoted=False),
      alias=TableAlias(
        t

In [54]:
ast.args

{'this': Schema(
   this=Table(
     this=Identifier(this=test, quoted=False)),
   expressions=[
     Identifier(this=id, quoted=False),
     Identifier(this=name, quoted=False)]),
 'by_name': None,
 'exists': None,
 'partition': None,
 'where': None,
 'expression': Select(
   expressions=[
     Column(
       this=Identifier(this=id, quoted=False),
       table=Identifier(this=t1, quoted=False)),
     Alias(
       this=Case(
         ifs=[
           If(
             this=GT(
               this=Coalesce(
                 this=Length(
                   this=Column(
                     this=Identifier(this=name1, quoted=False),
                     table=Identifier(this=t1, quoted=False))),
                 expressions=[
                   Literal(this=0, is_string=False)]),
               expression=Literal(this=0, is_string=False)),
             true=Column(
               this=Identifier(this=bname1, quoted=False),
               table=Identifier(this=t1, quoted=False)))],
      

In [29]:
ast.expression.expressions

[Column(
   this=Identifier(this=id, quoted=False),
   table=Identifier(this=t1, quoted=False)),
 Alias(
   this=Case(
     ifs=[
       If(
         this=GT(
           this=Coalesce(
             this=Length(
               this=Column(
                 this=Identifier(this=name1, quoted=False),
                 table=Identifier(this=t1, quoted=False))),
             expressions=[
               Literal(this=0, is_string=False)]),
           expression=Literal(this=0, is_string=False)),
         true=Column(
           this=Identifier(this=bname1, quoted=False),
           table=Identifier(this=t1, quoted=False)))],
     default=Column(
       this=Identifier(this=name2, quoted=False),
       table=Identifier(this=t2, quoted=False))),
   alias=Identifier(this=name, quoted=False))]

In [43]:
ast.expression.args

{'kind': None,
 'hint': None,
 'distinct': None,
 'expressions': [Column(
    this=Identifier(this=id, quoted=False),
    table=Identifier(this=t1, quoted=False)),
  Alias(
    this=Case(
      ifs=[
        If(
          this=GT(
            this=Coalesce(
              this=Length(
                this=Column(
                  this=Identifier(this=name1, quoted=False),
                  table=Identifier(this=t1, quoted=False))),
              expressions=[
                Literal(this=0, is_string=False)]),
            expression=Literal(this=0, is_string=False)),
          true=Column(
            this=Identifier(this=bname1, quoted=False),
            table=Identifier(this=t1, quoted=False)))],
      default=Column(
        this=Identifier(this=name2, quoted=False),
        table=Identifier(this=t2, quoted=False))),
    alias=Identifier(this=name, quoted=False))],
 'limit': None,
 'from': From(
   this=Table(
     this=Identifier(this=table1, quoted=False),
     alias=TableAlias(


ast.find_all

In [52]:
for pre in ast.expression.find_all(exp.Group):
    print(pre)


GROUP BY name


In [65]:
for pre in ast.find_all(exp.Join):
    print(pre)

LEFT JOIN table2 AS t2 ON t1.id = t2.id


In [63]:
for pre in ast.find_all(exp.Select):
    print(pre)

SELECT t1.id, CASE WHEN COALESCE(LENGTH(t1.name1), 0) > 0 THEN t1.bname1 ELSE t2.name2 END AS name FROM table1 AS t1 LEFT JOIN table2 AS t2 ON t1.id = t2.id WHERE NOT id IS NULL GROUP BY name


多sql情况

In [32]:
multi_sql='''
truncate table a;
insert into a(id,name)  select * from b;
'''

In [33]:
# 转换sql为 ast tree
# parse 会转换多个sql，为ast list
ast_list=sqlglot.parse(multi_sql)

'truncate table a' contains unsupported syntax. Falling back to parsing as a 'Command'.


In [40]:
for pre in ast_list:
    print(pre)
    print(type(pre))
    print(isinstance(pre,sqlglot.expressions.Insert))

TRUNCATE table a
<class 'sqlglot.expressions.Command'>
False
INSERT INTO a (id, name) SELECT * FROM b
<class 'sqlglot.expressions.Insert'>
True


In [7]:
sql='''
insert into a
select `(num)?+.+` from (
select *,row_number() over(partition by id order by id) as rn from b
) t where t.num=1
'''

In [8]:
sqlglot.parse(sql)

ParseError: Required keyword: 'expression' missing for <class 'sqlglot.expressions.JSONBContains'>. Line 3, Col: 15.
  
insert into a
select `(num)?[4m+[0m.+` from (
select *,row_number() over(partition by id order by id) as rn from b
) t where t.num=1
